In [16]:
import json
import web3

from web3 import Web3
from solc import compile_source
from web3.contract import ConciseContract

In [17]:
# Solidity source code
contract_source_code = '''
pragma solidity >0.4.0;

contract simplestorage {

  int public familyBalance;
  address public otherParentAddr;
  address public balanceSetterAddr;

  constructor(address parentAddr) public {
    familyBalance = 0;
    balanceSetterAddr = msg.sender;
    otherParentAddr = parentAddr;
  }

  function makeRecord(int x) public {
    if (x <= 0) {
        familyBalance = familyBalance + x;
    }
    else {
        if (msg.sender == balanceSetterAddr || msg.sender == otherParentAddr) {
            familyBalance = familyBalance + x;
        }
    }
  }

  function getBalance() public view returns (int retVal) {
    return familyBalance;
  }

}
'''
# Compiled source code
compiled_sol = compile_source(contract_source_code) 
contract_interface = compiled_sol['<stdin>:simplestorage']

In [18]:
NODE_ADDR = 'http://XXX.XXX.p2pify.com:8545'

In [19]:
# web3.py instance
w3 = Web3(Web3.HTTPProvider(NODE_ADDR))

In [20]:
# inject the poa compatibility middleware to the innermost layer
from web3.middleware import geth_poa_middleware
w3.middleware_stack.inject(geth_poa_middleware, layer=0)

In [21]:
contract_address = '0xa2B5e8e28e7E0187ee955b04C48f715ac43fcE08'

In [22]:
simplestorage = w3.eth.contract(
    address=contract_address,
    abi=contract_interface['abi'],
)

In [23]:
'Storage value:', simplestorage.functions.getBalance().call()

('Storage value:', 0)

In [24]:
tx_hash = simplestorage.functions.makeRecord(1000).transact()

ValueError: {'code': -32000, 'message': 'unknown account'}

In [25]:
w3.eth.defaultAccount = w3.eth.accounts[0]

In [26]:
tx_hash = simplestorage.functions.makeRecord(1000).transact()

In [27]:
tx_hash

HexBytes('0xdaf89458b01bd27464a584de7d5a062f97ea8d404714d8bec91ddf51ae1e11e5')

In [28]:
'Storage value:', simplestorage.functions.getBalance().call()

('Storage value:', 1000)